In [12]:
# # Install Java, Spark, and Findspark
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q http://www-us.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
# !tar xf spark-2.4.6-bin-hadoop2.7.tgz
# !pip install -q findspark

# # Set Environment Variables
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

# # Start a SparkSession
# import findspark
# findspark.init()
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
import pandas as pd
import re

In [13]:
df=pd.read_csv("Resources/data/tweetPredData.csv")
df

,Date,Matched Keywords,User,Source,Followers,Friends,Favorite,Tweet,predTweet,CleanedTweet,Prediction
0,08/05/2020,Biden,YourSiteingGoog,IFTTT,84,198,0,Breaking #FoxNews Alert : Bien scraps plans to...,breaking foxnews alert biden scraps plans trav...,"['break', 'foxnew', 'alert', 'bien', 'scrap', ...",Positive
1,08/05/2020,Trump,rbenton75nc,Twitter for Android,15,87,0,@JRubinBlogger Isn't the doctor who gave Trump...,not the doctor who gave trump the test the sam...,"['doctor', 'trump', 'test', 'doctor', 'tri', '...",Neutral
2,08/05/2020,Trump,Mrb18101,Twitter for iPhone,72,134,0,@WindsorMann The only friends Trump has had hi...,the only friends trump has had his entire life...,"['friend', 'trump', 'entir', 'life', 'one', 'b...",Positive
3,08/05/2020,Trump,mermansteve,Twitter Web App,5232,5664,0,Trump’s pick to manage public lands has four-d...,trumps pick manage public lands has fourdecade...,"['trump', 'pick', 'manag', 'public', 'land', '...",Negative
4,08/05/2020,Trump,biospherian,Twitter Web App,26,167,0,Because Trump is an asshole: http://t.co/Is5rk...,because trump an asshole,"['trump', 'asshol']",Neutral
...,...,...,...,...,...,...,...,...,...,...,...
59586,08/05/2020,Biden,AmericaFightHim,Twitter for Android,8238,8261,2,This is an attempt to siphon votes from Joe Bi...,this an attempt siphon votes from joe biden ka...,"['attempt', 'siphon', 'vote', 'bien', 'wayn', ...",Neutral
59587,08/05/2020,Trump,adechapmanlaw,Twitter for Android,116,353,0,"Ovid, Boris Johnson, Trump, climate crisis and...",covid boris johnson trump climate crisis and t...,"['covid', 'ori', 'johnson', 'trump', 'climat',...",Negative
59588,08/05/2020,Trump,bockmann_bronco,Twitter for iPhone,12,32,0,@dale8 Trump is a “thing”. Isn’t he? When will...,trump thing isnt he when will he finally just ...,"['trump', 'thing', 'isnt', 'final', 'away', 'l...",Positive
59589,08/05/2020,Trump,Maria17886334,Twitter for iPhone,0,1,0,"@JLBreezy00 Oh yea, TRUMP 2020!! OD BLESS AME...",oh yea trump god bless america and all its mil...,"['trump', 'bless', 'america', 'militari', 'pol...",Positive


In [14]:
cvec = CountVectorizer(stop_words='english', min_df=3, max_df=0.5, ngram_range=(1,2))
sf = cvec.fit_transform(df["CleanedTweet"])

In [15]:
# from pyspark.ml.feature import HashingTF, IDF,
transformer = TfidfTransformer()
transformed_weights = transformer.fit_transform(sf)
weights = np.asarray(transformed_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': cvec.get_feature_names(), 'weight': weights})

In [16]:
weights_df.sort_values(by='weight', ascending=False).head(20)

,term,weight
2925,bien,0.023112
23157,presid,0.012432
35216,vote,0.011771
9105,donald,0.011109
16961,like,0.010852
9111,donald trump,0.010730
21718,peopl,0.009070
14939,interview,0.008763
15840,know,0.007626
30624,think,0.007490


In [38]:
weights_df.loc[weights_df["term"]=="community"]

,term,weight
